## pyRaceRoom Jupyter notebook 
- RaceHash: a unique identifier for each race
- StartPosition: the starting position of the player in the race
- TrackLayoutName: the name of the layout of the track
- CarName: the name of the player's car
- FinishPosition: the finishing position of the player in the race
- RatingAfter: the player's rating after the race
- ReputationAfter: the player's reputation after the race
- PlayersCount: the number of players in the race
- IncidentPoints: the number of incident points the player received during the race
- RatingChange: the change in the player's rating after the race
- RaceFinishTime: the time it took for the player to finish the race
- TrackId: a unique identifier for the track
- TrackFullName: the full name of the track
- ReputationChange: the change in the player's reputation after the race
- LapsCount: the number of laps in the race
- MinSectorTimesSum: the sum of the minimum sector times for all the player's valid laps
- ValidLapsCount: the number of valid laps completed by the player
- QualiLapsCount: the number of laps completed by the player during qualifying
- QualiMinSectorTimesSum: the sum of the minimum sector times for all the player's valid laps during qualifying
- QualiValidLapsCount: the number of valid laps completed by the player during qualifying
- TrackName: the name of the track

Raceroom user (Steam username)

In [18]:
username = 'diogorsousa'

Import career page

In [19]:
import requests

# save html
def save_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)

# open html
def open_html(path):
    with open(path, 'rb') as f:
        return f.read()

url = 'https://game.raceroom.com/users/' + username + '/career'
r = requests.get(url)
save_html(r.content, 'career')

In [ ]:
import requests
from bs4 import BeautifulSoup
from lxml import etree
import json
import pandas as pd

html = open_html('career')
soup = BeautifulSoup(html, "html.parser")
dom = etree.HTML(str(soup))

# Find the div with class "career"
careerPage = soup.find('div', {'class': 'careerPage'})

# Extract the data-entries attribute and decode the JSON data
data_entries = careerPage['data-entries']
data = json.loads(data_entries)

# Create a list of dictionaries containing the data for each race
races_data = []
for race in data:
    race_data = {}
    race_data['RaceHash'] = race['RaceHash']
    race_data['StartPosition'] = race['StartPosition']
    race_data['TrackLayoutName'] = race['TrackLayoutId']['Name']
    race_data['CarName'] = race['Cars'][0]['Name']
    race_data['FinishPosition'] = race['FinishPosition']
    race_data['RatingAfter'] = race['RatingAfter']
    race_data['ReputationAfter'] = race['ReputationAfter']
    race_data['PlayersCount'] = race['PlayersCount']
    race_data['IncidentPoints'] = race['IncidentPoints']
    race_data['RatingChange'] = race['RatingChange']
    race_data['RaceFinishTime'] = race['RaceFinishTime']

    # Get more details from API
    api_url = f'https://game.raceroom.com/multiplayer/results/{race["RaceHash"]}'
    api_response = requests.get(api_url)
    api_data = api_response.json()
    
    # Extract additional data
    race_data['TrackId'] = api_data['GetMpRaceResultResult']['TrackId']['Id']
    race_data['TrackFullName'] = api_data['GetMpRaceResultResult']['TrackId']['Name']

    my_position = -1

    # Encontre a posição do seu username na lista de resultados da corrida
    for i, result in enumerate(api_data['GetMpRaceResultResult']['RaceResult']):
        if result['Username'] == username:
            my_position = i 
            break
    
    # Get race results
    race_results = api_data['GetMpRaceResultResult']['RaceResult']

    # Get user's race result
    if my_position >= 0 and my_position < len(race_results):
        user_race_result = race_results[my_position]
        race_data['ReputationChange'] = user_race_result['ReputationChange']

        # Get user's lap data
        laps_data = user_race_result['Laps']

        # Count number of laps and sum of sector times
        lap_count = 0
        sector_times = []
        min_sector_times_sum = float('inf')
        valid_lap_count = 0

        # Iterate over the laps to count laps, sum sector times, and find the minimum sector time sum
        for lap in laps_data:
            lap_count += 1

            if lap['Valid'] == True:
                valid_lap_count += 1
                if lap['Time'] > 0 and lap['Time'] < min_sector_times_sum:
                    min_sector_times_sum = lap['Time']

        # Add lap data to race data
        race_data['LapsCount'] = lap_count
        race_data['MinSectorTimesSum'] = min_sector_times_sum
        race_data['ValidLapsCount'] = valid_lap_count
        race_data['CarName'] = user_race_result['CarClass']['Name']
    else:
        user_race_result = None  # ou outro valor padrão que faça sentido para o seu caso

    #Get quali results
    quali_results = api_data['GetMpRaceResultResult']['QualiResult']

    # Get user's quali result
    user_quali_result = None
    for result in quali_results:
        if result['Username'] == username:
            user_quali_result = result
            break

    if user_quali_result is None:
        raise ValueError(f"Username {username} not found in quali results")

    # Get user's lap data
    laps_data = user_quali_result['Laps']

    # Count number of laps and sum of sector times
    lap_count = 0
    sector_times = []
    sector_time_sum = 0
    min_sector_times_sum = float('inf')
    valid_lap_count = 0

    # Iterate over the laps to count laps, sum sector times, and find the minimum sector time sum
    for lap in laps_data:
        lap_count += 1

        if lap['Valid'] == True:
            valid_lap_count += 1
            if lap['Time'] > 0 and lap['Time'] < min_sector_times_sum:
                min_sector_times_sum = lap['Time']

    race_data['QualiLapsCount'] = lap_count
    race_data['QualiMinSectorTimesSum'] = min_sector_times_sum
    race_data['QualiValidLapsCount'] = valid_lap_count 

    print(race_data)
    races_data.append(race_data)

# Create a pandas DataFrame from the list of dictionaries
df = pd.DataFrame(races_data)

# Sort the DataFrame by RaceFinishTime in descending order
df = df.sort_values('RaceFinishTime', ascending=False)

# Convert timestamp to date time
df['RaceFinishTime'] = pd.to_datetime(df['RaceFinishTime'], unit='s')

# Concatenate Track name and layout name
df['TrackName'] = df['TrackFullName'] + ' - ' + df['TrackLayoutName']


In [84]:
# Save DataFrame to binary pandas file
df.to_pickle(username + '.pkl')

In [86]:
df = pd.read_pickle(username + '.pkl')

FinishPosition and mean of StartPosition

In [87]:
# Group the dataframe by FinishPosition and calculate the mean of StartPosition for each group
result = df.groupby('FinishPosition')['StartPosition'].mean().round(1)

# Set the column header name
result.name = 'MeanStartPosition'

# Display the result with column headers
print(result.to_frame().reset_index().to_string(index=False))

 FinishPosition  MeanStartPosition
              1                1.3
              2                2.2
              3                2.5
              4                3.5
              5                3.5
              6                3.0
              7                3.0
              8                3.0
              9                5.5
             12               10.5
             15                8.0
             18               16.0
             24               11.0


In [88]:
grouped = df.groupby('StartPosition')[['RatingChange', 'ReputationChange']].mean()

# rename the columns
grouped = grouped.rename(columns={'RatingChange': 'MeanRatingChange', 'ReputationChange': 'MeanReputationChange'})

# display the resulting dataframe
print(grouped)

               MeanRatingChange  MeanReputationChange
StartPosition                                        
1                      1.318231              0.182462
2                      0.960353              0.064735
3                     -0.010789             -0.050474
4                     -0.909600             -0.140800
5                     -1.474600             -0.015000
7                      3.189000             -0.087500
11                   -19.950000             -0.656000
13                    -6.854000             -1.843000
16                    -9.167000             -0.402000
17                    -1.488000             -0.384000


Car analysis

In [89]:
# group by car and track, calculate average rating change, reputation change, and mean finish position
result = df.groupby(['CarName']).agg({'RatingChange': 'mean', 'ReputationChange': 'mean', 'FinishPosition': 'mean', 'RaceHash': 'count'})

# rename count column to 'NumRaces'
result = result.rename(columns={'RaceHash': 'NumRaces'})

# sort by number of races in descending order
result = result.sort_values(by='NumRaces', ascending=False)

# print the result
print(result)

                             RatingChange  ReputationChange  FinishPosition  \
CarName                                                                       
Tatuus F4 Cup                    0.322093          0.105187        3.520000   
BMW M235i Racing Cup            -0.218846         -0.374154        3.461538   
GTR 3                            3.885667          0.077667        2.666667   
GTR 4                           -8.167667         -0.443333        6.333333   
NSU TTS Cup                     -0.730500         -0.203500        3.000000   
Porsche Carrera Cup Classic      0.509000         -0.711000        1.000000   

                             NumRaces  
CarName                                
Tatuus F4 Cup                      75  
BMW M235i Racing Cup               13  
GTR 3                               6  
GTR 4                               3  
NSU TTS Cup                         2  
Porsche Carrera Cup Classic         1  


Track analysis

In [90]:
import numpy as np

# assuming the dataframe is called 'df'
min_times = df.groupby(['TrackName', 'CarName'])[['QualiMinSectorTimesSum', 'MinSectorTimesSum']].min()
min_times = min_times.rename(columns={'QualiMinSectorTimesSum': 'BestQualiLap', 'MinSectorTimesSum': 'BestRaceLap'})

# define function to convert milliseconds to mm:ss format
def convert_to_mins(x):
    if pd.isna(x):
        return ''
    if x == np.inf or x == -np.inf:
        return 'N/A'
    if abs(x) > 10**7:
        return 'N/A'
    mins = int(x / 60000)
    secs = int((x / 1000) % 60)
    millis = int(x % 1000)
    return '{:02d}:{:02d}.{:03d}'.format(mins, secs, millis)

# apply the conversion function to the columns
min_times['BestQualiLap'] = min_times['BestQualiLap'].apply(convert_to_mins)
min_times['BestRaceLap'] = min_times['BestRaceLap'].apply(convert_to_mins)

num_races = df.groupby(['TrackName', 'CarName'])[['RaceHash']].count()
total_laps = df.groupby(['TrackName', 'CarName'])[['LapsCount']].sum()
valid_laps = df.groupby(['TrackName', 'CarName'])[['ValidLapsCount']].sum()

result = min_times.join(num_races).join(total_laps).join(valid_laps)
result['ValidLapsPct'] = (result['ValidLapsCount'] / result['LapsCount'] * 100).round(2)

# reset the index to convert the MultiIndex to regular columns
result = result.reset_index()

# rename columns to be more descriptive
result = result.rename(columns={'TrackName': 'Track', 'CarName': 'Car', 'BestQualiLap': 'Quali Lap Time', 
                                'BestRaceLap': 'Race Lap Time', 'RaceHash': 'Number of Races', 
                                'LapsCount': 'Total Laps', 'ValidLapsCount': 'Valid Laps',
                                'ValidLapsPct': 'Valid Laps %'})

# display the resulting dataframe
print(result)
result.to_csv('result.csv', index=True)

                                                Track  \
0                            Brands Hatch Indy - Indy   
1                 Circuit Zandvoort 2019 - Grand Prix   
2        Daytona International Speedway - Road Course   
3                         Hockenheimring - Grand Prix   
4                         Hockenheimring - Grand Prix   
5                                  Imola - Grand Prix   
6                             Interlagos - Grand Prix   
7                          Monza Circuit - Grand Prix   
8               Nürburgring - Grand Prix Fast Chicane   
9                       Portimao Circuit - Grand Prix   
10                      Portimao Circuit - Grand Prix   
11       Red Bull Ring Spielberg - Grand Prix Circuit   
12                     Red Bull Ring Spielberg - Moto   
13                     Spa-Francorchamps - Grand Prix   
14                        Suzuka Circuit - Grand Prix   
15                        Suzuka Circuit - Grand Prix   
16                       Suzuka